In [1]:
from DCGAN import DCGAN
from fid import calculate_frechet_distance, compute_statistics_of_path, calculate_frechet_distance
from inception import InceptionV3
import os
import numpy as np
import torch

%load_ext autoreload
%autoreload 2

/home/elias/.virtualenvs/torch/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Number of channels in the training images. For color images this is 3
num_channels = 3

# Size of z latent vector (i.e. size of generator input)
latent_dim = 128

# Size of feature maps in generator
num_generator_features = 128

# Size of feature maps in discriminator
num_discriminator_features = 64

NUM_WORKERS = int(os.cpu_count() - 1)
BATCH_SIZE = 6 

IMAGE_SIZE = 512

device = "cpu"

dataset_path = "../data/celeba/img_align_celeba/img_align_celeba/"

dataset_path = "../data/art_dataset/resized/resized/"
dataset_statistics_path = "/".join(dataset_path.split("/")[:-1]) + "/inception_statistics.npz"

In [3]:
model = DCGAN(
    num_channels=num_channels,
    latent_dim=latent_dim,
    num_generator_features=num_discriminator_features,
    num_discriminator_features=num_discriminator_features,
).to(device)

In [4]:
inception_model = InceptionV3().to(device)

## Compute statistics for dataset

In [5]:
m1, s1 = compute_statistics_of_path(dataset_path, inception_model, BATCH_SIZE, 2048, device, 12)
print(m1, s1)


ValueError: batch_size should be a positive integer value, but got batch_size=0

In [ ]:

np.savez(dataset_statistics_path, m=m1, s=s1)

FileNotFoundError: [Errno 2] No such file or directory: '../data/art_dataset/resized/resized/inception_statistics.npz'

In [ ]:
dataset_statistics_file = np.load(dataset_statistics_path)
m = dataset_statistics_file["m"]
s = dataset_statistics_file["s"]
print(m.shape)
print(s.shape)

(2048,)
(2048, 2048)


## Compute statistics for batch of predictions

In [ ]:
z = torch.randn(BATCH_SIZE, latent_dim, 1, 1).to(device)
generated_images = model(z)
print(z.shape)

torch.Size([6, 128, 1, 1])


In [ ]:
pred = inception_model(generated_images)[0]

In [ ]:
resized_pred = pred.squeeze().detach().cpu().numpy()
mu2 = np.mean(resized_pred, axis=0)
sigma = np.cov(resized_pred, rowvar=False)

In [ ]:
print(mu2.shape)
print(sigma.shape)

(2048,)
(2048, 2048)


In [ ]:
calculate_frechet_distance(mu2, sigma, m, s)

545.1638588031259

In [4]:
from fid import FID
ds_fid = FID("celeba", IMAGE_SIZE, BATCH_SIZE)

ds_fid.compute_statistics_of_path()

/home/elias/dev/personal/art-gan-project/data/celeba/images


  0%|          | 46/33767 [00:13<2:53:53,  3.23it/s]